# Segmenting and Clustering the Neighborhoods in Toronto
## Data Science Capstone Week 3


## Part 1

In [1]:
import pandas as pd
import numpy as np
import wget

In [2]:
#read in the data from wikipedia
wiki  = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#only take the Postal Code, Borough, and Neighbourhood data
df = wiki[0]


In [3]:
#remove rows that are not assigned a borough
df['Borough'].replace("Not assigned", np.nan, inplace = True)
df.dropna(subset = ["Borough"], axis = 0, inplace = True)

#set empty neighbourhood names to bourough name
df['Neighbourhood'].replace("Not assigned", df['Borough'], inplace = True)

#reset index
df.reset_index(drop = True, inplace = True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
#shape of the dataframe
df.shape

(103, 3)

## Part 2

In [5]:
#download geosspatial data
url = 'http://cocl.us/Geospatial_data'
coords = wget.download(url)

#load data into pandas dataframe
geo_cords = pd.read_csv(coords)

#set the postal code as index
geo_cords.set_index('Postal Code',inplace = True)
geo_cords

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [6]:
#get the relevant postal codes
postal_codes = df[['Postal Code']]

#merges data frames to get only rows with relevant postal codes
common = postal_codes.merge(geo_cords,on=['Postal Code','Postal Code'])

#add latitude and lonitude to original frame
df[["Latitude", "Longitude"]] = common[["Latitude", "Longitude"]]

df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3

In [7]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

done
done

# All requested packages already installed.

done

# All requested packages already installed.



# Data Visualization and Analysis

### Only want to use the Boroughs with the word 'Toronto' in the name

In [8]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Use geopy to get coordinates of Toronto

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude+.03
longitude = location.longitude


In [10]:
#Create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add postal code markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
        
map_toronto  

In [11]:
import yaml
#Get foursquare credentials
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

CLIENT_ID = cfg['api_creds']['client_id'] # your Foursquare ID
CLIENT_SECRET = cfg['api_creds']['client_secret']  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


## Find all the venues near each postal code

In [12]:
#set max number of venues returned for each postal code
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=toronto_data['Postal Code'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


## Number of venues in each postal code


In [14]:
toronto_venues.groupby('Postal Code').count()

,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,5,5,5,5,5,5
M4K,41,41,41,41,41,41
M4L,21,21,21,21,21,21
M4M,41,41,41,41,41,41
M4N,3,3,3,3,3,3
M4P,8,8,8,8,8,8
M4R,18,18,18,18,18,18
M4S,37,37,37,37,37,37
M4T,2,2,2,2,2,2


## Unique Categories

In [15]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## Analyzing Postal Codes


In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Group rows by postal code and take the mean of the frequency of occurrence of each category

In [17]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.20000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.024390,0.000000,...,0.000000,0.000000,0.02439,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.024390
2,M4L,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.048780,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.024390,0.00,0.024390
4,M4N,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.055556,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.055556
7,M4S,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.027027,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.50000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.062500,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.062500,0.000000,0.00,0.000000


## Finding the top 10 venues in each postal code

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_venues_sorted = pd.DataFrame(columns=columns)
postal_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    postal_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postal_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Asian Restaurant,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Ice Cream Shop,Bubble Tea Shop,Spa,Japanese Restaurant,Juice Bar
2,M4L,Park,Sandwich Place,Brewery,Liquor Store,Board Shop,Burrito Place,Restaurant,Italian Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,M4M,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Food,Pet Store,Park
4,M4N,Park,Bus Line,Swim School,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Clustering Postal Codes


In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 0, 3, 0], dtype=int32)

In [23]:
# add clustering labels
postal_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each postal code
toronto_merged = toronto_merged.join(postal_venues_sorted.set_index('Postal Code'), on='Postal Code', how = 'right')

In [24]:
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Café,Park,Restaurant,Breakfast Spot,Theater,Yoga Studio,Farmers Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Persian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Pizza Place,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,American Restaurant,Gastropub,Farmers Market,Seafood Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run


## Visualize the results

In [26]:
type(toronto_merged)

pandas.core.frame.DataFrame

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Café,Park,Restaurant,Breakfast Spot,Theater,Yoga Studio,Farmers Market
1,Downtown Toronto,0,Coffee Shop,Diner,Yoga Studio,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Persian Restaurant
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Pizza Place,Diner
3,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,American Restaurant,Gastropub,Farmers Market,Seafood Restaurant
4,East Toronto,0,Asian Restaurant,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run
5,Downtown Toronto,0,Coffee Shop,Farmers Market,Bakery,Café,Restaurant,Cheese Shop,Seafood Restaurant,Beer Bar,Cocktail Bar,Museum
6,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Restaurant,Salad Place,Department Store,Japanese Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Diner,Candy Store,Baby Store,Coffee Shop,Nightclub
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Hotel,Bar,Steakhouse,Thai Restaurant,Lounge
9,West Toronto,0,Pharmacy,Bakery,Brewery,Bank,Supermarket,Café,Pizza Place,Middle Eastern Restaurant,Art Gallery,Music Venue
